
Lab | BabyAGI with agent

Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats You can't get this wrong!

You would need the OpenAI API KEY and the SerpAPI KEY to run this lab.
BabyAGI with Tools

This notebook builds on top of baby agi, but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information


In [1]:
# ============================================================
# INSTALL REQUIRED PACKAGES
# ============================================================
# We install a specific set of LangChain packages that work together.
# google-search-results is required for SerpAPI search.
!pip install -q langchain langchain-openai langchain-core langchain-community \
                  langchain-experimental langchain-classic python-dotenv \
                  faiss-cpu google-search-results


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# ============================================================
# IMPORTS
# ============================================================

import os
import sys
import getpass
from dotenv import load_dotenv, find_dotenv

# OpenAI + Embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Tools + Document schema
from langchain_core.tools import Tool
from langchain_core.documents import Document

# BabyAGI (experimental LangChain agent)
from langchain_experimental.autonomous_agents import BabyAGI

# Vector store
from langchain_community.vectorstores import FAISS

# Search tool
from langchain_community.utilities import SerpAPIWrapper

# Classic agent wrapper
from langchain_classic.agents import initialize_agent, AgentType


In [3]:
# ============================================================
# LOAD ENVIRONMENT VARIABLES
# ============================================================
# If .env file exists, load it. Otherwise user inputs API keys manually.

load_dotenv(find_dotenv())

# Enter API keys interactively (safer for public notebooks)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")
os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your SERPAPI_API_KEY: ")


Enter your OPENAI_API_KEY: ··········
Enter your SERPAPI_API_KEY: ··········


In [27]:
# ============================================================
# INITIALIZE LLM + EMBEDDINGS
# ============================================================
# ChatOpenAI uses the OpenAI Chat Completions API.
# model="gpt-3.5-turbo" is low-cost and works well for BabyAGI examples.

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Embeddings used by FAISS for vector search
embeddings_model = OpenAIEmbeddings()


In [34]:
# ============================================================
# CREATE VECTORSTORE WITH EXAMPLE DOCUMENTS
# ============================================================
# BabyAGI uses a vector database to store and recall tasks.
# Here we store generic "task descriptions" as example documents.

documents = [
    Document(page_content="Create comprehensive plans", metadata={"task": "Planning"}),
    Document(page_content="Research information", metadata={"task": "Research"}),
    Document(page_content="Generate schedules", metadata={"task": "Scheduling"}),
    Document(page_content="Create checklists",  metadata={"task": "Organization"}),
]

# Build the FAISS vector index
vectorstore = FAISS.from_documents(documents, embeddings_model)


In [35]:
# ============================================================
# DEFINE TOOLS (FUNCTIONS BABYAGI CAN USE)
# ============================================================
# Each tool wraps an LLM or external call (like SerpAPI).
# Tools allow BabyAGI to perform actions.

# ---- 1. PLANNING TOOL ----
def create_plan(objective: str) -> str:
    """Generate a detailed plan for a given objective."""
    response = llm.invoke(
        f"Create a detailed step-by-step plan for: {objective}.\n"
        f"Include milestones, timeline, and required resources."
    )
    return response.content

# ---- 2. SEARCH TOOL ----
def research_topic(query: str) -> str:
    """Run a SerpAPI search or return a fallback message."""
    try:
        search = SerpAPIWrapper()
        return search.run(query)
    except:
        return f"Basic research result for: {query}"

# ---- 3. SCHEDULE TOOL ----
def generate_schedule(objective: str) -> str:
    """Generate a multi-day schedule for an objective."""
    response = llm.invoke(
        f"Create a day-by-day schedule for: {objective}"
    )
    return response.content

# ---- 4. CHECKLIST TOOL ----
def create_checklist(objective: str) -> str:
    """Create a clean and organized checklist."""
    response = llm.invoke(
        f"Create a detailed checklist for: {objective}"
    )
    return response.content

# ---- PACKAGE TOOLS INTO LANGCHAIN TOOL OBJECTS ----
tools = [
    Tool(name="Research",   func=research_topic,   description="Search for information online."),
    Tool(name="Planner",    func=create_plan,      description="Create detailed plans."),
    Tool(name="Scheduler",  func=generate_schedule, description="Generate multi-day schedules."),
    Tool(name="Checklist",  func=create_checklist, description="Create organized checklists."),
]


In [36]:
# ============================================================
# CREATE TRADITIONAL AGENT (OPTIONAL)
# ============================================================
# This is not BabyAGI yet – this is a reactive agent that can use tools.
# BabyAGI will call tools internally, but we show both for clarity.

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5
)


In [40]:
# ============================================================
# CREATE BABYAGI INSTANCE
# ============================================================
# BabyAGI uses:
#  - The LLM
#  - A vectorstore to store tasks/results
#  - Automatically generated tasks
#  - max_iterations to prevent infinite loops

baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    verbose=False,
    max_iterations=3
)


In [23]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")



🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> Plan a 5-day trip to Italy including accommodations, activities, and budget

************************************************************
OBJECTIVE: Plan a 5-day trip to Italy including accommodations, activities, and budget
************************************************************



In [24]:
# ============================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****

1. Research and book accommodations for each night of the trip
2. Research and plan activities for each day of the trip
3. Create a budget for the trip, including accommodations, activities, transportation, and meals
4. Schedule transportation to and from Italy
5. Organize all travel documents and necessary information for the trip
6. Pack necessary items for the trip
7. Confirm all bookings and reservations before departure
8. Create a daily itinerary for each day of the trip
9. Keep track of expenses during the trip
10. Enjoy and make the most of the 5-day trip to Italy!

*****TASK LIST*****

1: Research and book tickets for any activities or attractions that require advance booking
2: Research and book transportation within Italy for each day of the trip
3: Research and plan dining options for each day of the trip, including local restaurants and cafes
4: Create a packing list

following are the exmaples you can try:
# Example 1: Travel Planning
"Plan a 5-day trip to Italy including accommodations, activities, and budget"

# Example 2: Project Management
"Create a launch plan for my new product including timeline, marketing, and resources"

# Example 3: Personal Planning
"Plan a healthy meal prep for the next 2 weeks with shopping list and recipes"

# Example 4: Event Planning
"Organize a wedding for 100 guests in 3 months including venue, catering, and timeline"

# Example 5: Learning
"Create a 30-day plan to learn Python programming from scratch"

In [43]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")


🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> Create a 30-day plan to learn Python programming from scratch

************************************************************
OBJECTIVE: Create a 30-day plan to learn Python programming from scratch
************************************************************



In [44]:
#===========================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****

Day 1: Introduction to Python
- Install Python on your computer
- Learn about Python syntax and basic data types
- Practice writing simple Python scripts

Day 2: Variables and Data Structures
- Understand how to use variables in Python
- Learn about different data structures like lists, tuples, and dictionaries
- Practice working with data structures in Python

Day 3: Control Flow
- Learn about conditional statements (if, elif, else) in Python
- Understand loops (for and while) in Python
- Practice using control flow in Python scripts

Day 4: Functions
- Learn how to define and call functions in Python
- Understand the concept of scope in Python
- Practice writing and using functions in Python

Day 5: Modules and Packages
- Learn about modules and how to import them in Python
- Understand the concept of packages and how to install them
- Practice using modules and packages in Pyt

In [41]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")


🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> Organize a wedding for 100 guests in 3 months including venue, catering, and timeline

************************************************************
OBJECTIVE: Organize a wedding for 100 guests in 3 months including venue, catering, and timeline
************************************************************



In [42]:
#===========================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****

1. Secure a venue for the wedding ceremony and reception
2. Hire a caterer and plan the menu incorporating seasonal produce
3. Create a timeline for the wedding day including ceremony, reception, and any other events
4. Send out save the date cards and invitations to guests
5. Arrange for transportation for guests if needed
6. Book a photographer and videographer for the event
7. Select and order wedding attire for the bride, groom, and wedding party
8. Arrange for music and entertainment for the reception
9. Plan and order decorations for the venue
10. Organize accommodations for out-of-town guests
11. Arrange for any necessary permits or licenses for the event
12. Create a seating chart for the reception
13. Plan and order wedding favors for guests
14. Coordinate with vendors to ensure everything is in place for the big day
15. Create a backup plan in case of any unforeseen cir

In [38]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")


🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> Plan a healthy meal prep for the next 2 weeks with shopping list and recipes

************************************************************
OBJECTIVE: Plan a healthy meal prep for the next 2 weeks with shopping list and recipes
************************************************************



In [39]:
#===========================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****

1. Research healthy meal prep recipes for breakfast, lunch, and dinner for the next 2 weeks.
2. Create a meal plan schedule for each day, including portion sizes and nutritional information.
3. Compile a shopping list of all ingredients needed for the meal prep.
4. Organize the shopping list by category (e.g. produce, proteins, grains, etc.).
5. Schedule time for grocery shopping and meal prep for the next 2 weeks.
6. Prepare and cook meals according to the meal plan schedule.
7. Store meals in portioned containers for easy access throughout the week.
8. Monitor and track progress of meal prep to ensure adherence to the plan.
9. Adjust meal plan as needed based on feedback and results.
10. Reflect on the meal prep experience and make notes for future planning.

*****TASK LIST*****

1: Research and incorporate any dietary restrictions or preferences into the meal plan.
2: Research

In [13]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")


🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> Plan a 5-day trip to Italy including accommodations, activities, and budget

************************************************************
OBJECTIVE: Plan a 5-day trip to Italy including accommodations, activities, and budget
************************************************************



In [26]:
#===========================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Research and book a cooking class in Bologna to learn how to make traditional Italian dishes from local chefs.
2: Research and book tickets for a wine tasting experience in Tuscany to enjoy the region's famous wines and picturesque vineyards.
3: Research and book tickets for a guided tour of the Leaning Tower of Pisa to learn about its history and architecture.
4: Research and book a guided tour of the Amalfi Coast to explore the stunning coastal towns and enjoy breathtaking views of the Mediterranean Sea.
5: Research and book tickets for a guided tour of the Colosseum in Rome to learn about its history and significance in ancient Roman culture.
6: Research and book tickets for a guided tour of the Vatican City in Rome to explore its famous museums, St. Peter's Basilica, and the Sistine Chapel.
7: Research and book a gondola ride in Venice to experience the city's iconic canals and historic landmarks from the water.
8: Research and plan a visit to the Cinque Te

In [32]:
# ============================================================
# USER INTERFACE / INTERACTION
# ============================================================

print("\n" + "="*60)
print("🤖 AI PLANNING ASSISTANT (BabyAGI)")
print("="*60)

print("\nExamples you can try:")
print("  • Plan a 5-day trip to Italy")
print("  • Create a meal prep plan for next week")
print("  • Organize a product launch")
print("  • Plan a home renovation project")

user_objective = input("\nWhat would you like help planning?\n> ").strip()

# Default if user presses Enter
if not user_objective:
    user_objective = "Plan a 5-day trip to Italy"

print(f"\n{'*'*60}")
print(f"OBJECTIVE: {user_objective}")
print(f"{'*'*60}\n")


🤖 AI PLANNING ASSISTANT (BabyAGI)

Examples you can try:
  • Plan a 5-day trip to Italy
  • Create a meal prep plan for next week
  • Organize a product launch
  • Plan a home renovation project

What would you like help planning?
> Create a launch plan for my new product including timeline, marketing, and resources

************************************************************
OBJECTIVE: Create a launch plan for my new product including timeline, marketing, and resources
************************************************************



In [33]:
#===========================================================
# RUN BABYAGI
# ============================================================
try:
    result = baby_agi.invoke({"objective": user_objective})

    print(f"\n{'*'*60}")
    print("✅ FINAL PLANNING OUTPUT")
    print(f"{'*'*60}\n")
    print(result)

except Exception as e:
    print("\n❌ ERROR OCCURRED:", e)
    print("\nTROUBLESHOOTING:")
    print("1. Are your API keys correct?")
    print("2. Do you have internet access?")
    print("3. Is SerpAPI installed? Run: pip install google-search-results")



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****

1. Define target market and create buyer personas
2. Develop a marketing strategy including social media, email campaigns, and advertising
3. Create a launch timeline with key milestones and deadlines
4. Secure necessary resources such as budget, team members, and materials
5. Coordinate with vendors and partners for any collaborations or promotions
6. Conduct product testing and gather feedback for improvements
7. Prepare promotional materials such as press releases, product photos, and website updates
8. Train sales team on product features and benefits
9. Monitor and track progress of launch plan to ensure everything is on schedule
10. Evaluate the success of the launch and make adjustments as needed.

*****TASK LIST*****

1: Identify key influencers and potential brand ambassadors for product promotion
2: Develop a pricing strategy and promotional offers for the launch
3: Cre